# Comment charger des fichiers CSV depuis un stage vers Snowflake Notebooks 📁

Dans cet exemple, nous allons montrer comment charger un fichier CSV depuis un stage et créer une table avec Snowpark.

Commençons par utiliser la commande `get_active_session` afin d’obtenir la variable de contexte de session pour travailler avec Snowpark comme suit :

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()
# Add a query tag to the session. This helps with troubleshooting and performance monitoring.
session.query_tag = {"origin":"sf_sit-is", 
                     "name":"notebook_demo_pack", 
                     "version":{"major":1, "minor":0},
                     "attributes":{"is_quickstart":1, "source":"notebook", "vignette":"csv_from_s3"}}
print(session)

## Créer un stage externe

Nous allons ensuite créer un stage externe qui référence des fichiers de données stockés en dehors de Snowflake.
Dans cet exemple, les données se trouvent dans un bucket S3.

In [ ]:

CREATE DATABASE IF NOT EXISTS CITIBIKE;

USE DATABASE CITIBIKE;

USE SCHEMA PUBLIC;

CREATE STAGE IF NOT EXISTS CITIBIKE_STAGE 
	URL = 's3://logbrain-datalake/datasets/citibike-trips-csv/';

## Examiner les fichiers présents dans le stage

Regardons les fichiers disponibles dans le stage.

In [ ]:
LS @CITIBIKE_STAGE;

## Charger le fichier CSV avec Snowpark

Nous pouvons utiliser Snowpark DataFrameReader pour lire le fichier CSV.

En utilisant l’option `infer_schema = True`, Snowflake déduira automatiquement le schéma à partir des types de données présents dans le fichier CSV, ce qui évite de devoir le définir à l’avance.

In [ ]:
# Create a DataFrame that is configured to load data from the CSV file.
df = session.read.options({"infer_schema":True}).csv('@CITIBIKE_STAGE/trips_2018_0_0_0.csv.gz')

In [ ]:
df

## Travailler avec le DataFrame Snowpark

Maintenant que les données sont chargées dans un DataFrame Snowpark, nous pouvons les manipuler à l’aide de l’API Snowpark DataFrame.

Par exemple, nous pouvons calculer des statistiques descriptives sur les colonnes.

In [ ]:
df.describe()

In [ ]:
df.dtypes

### Creation d'un Schema de données

In [ ]:
from snowflake.snowpark.types import StructType, StructField, IntegerType, StringType, FloatType, TimestampType

trips_schema = StructType([
    StructField("tripduration", IntegerType()),
    StructField("starttime", TimestampType()),
    StructField("stoptime", TimestampType()),
    StructField("start_station_id", IntegerType()),
    StructField("start_station_name", StringType()), 
    StructField("start_station_latitude", FloatType()),
    StructField("start_station_longitude", FloatType()),
    StructField("end_station_id", IntegerType()), 
    StructField("end_station_name", StringType()),
    StructField("end_station_latitude", FloatType()),
    StructField("end_station_longitude", FloatType()),  
    StructField("bikeid", IntegerType()),
    StructField("membership_type", StringType()), 
    StructField("usertype", StringType()), 
    StructField("birth_year", IntegerType()), 
    StructField("gender", IntegerType())
    ])

## Utilisation d'un schema
Pour pouvoir avoir les noms des colonnes dans le dataframe on va utiliser trips_schema lors du charegement de données.
On va aussi définir des options de formats pour pouvoir lire le fichier correctement.

In [ ]:
# re Create a DataFrame that is configured to load data from the CSV file with schema.
df = session.read.options({"infer_schema":True,"field_delimiter": ",", "skip_header": 1, "field_optionally_enclosed_by" : '\042' , "null_if" : ('')}).schema(trips_schema).csv('@CITIBIKE_STAGE/trips_2018_0_0_0.csv.gz')

In [ ]:
df

### Créer un file fromat:

On peut aussi créer le file formation séparement et l'utiliser lors du chargement des données.

In [ ]:
format = session.sql("create or replace file format csv_format type = 'CSV' field_delimiter = ',' record_delimiter = '\n' skip_header = 1  field_optionally_enclosed_by = '\042' null_if = (''); ").collect()

df = session.read.option("format_name", "csv_format").schema(trips_schema).csv('@CITIBIKE_STAGE/trips_2018_0_0_0.csv.gz')

In [ ]:
df

## Écrire le DataFrame dans une table Snowflake

Nous pouvons enregistrer le DataFrame dans une table appelée `APP_ORDER` puis l’interroger en SQL.

In [ ]:
df.write.mode("overwrite").save_as_table("TRIPS")

In [ ]:
-- Preview the newly created TRIPS table
SELECT * from TRIPS;

## Relire la table dans Snowpark

Nous pouvons relire la table dans Snowpark en utilisant la syntaxe `session.table`.

In [ ]:
df = session.table("TRIPS")
df

## Continuer le traitement des données

À partir de là, vous pouvez continuer à interroger et traiter les données.

In [ ]:
df.groupBy('"START_STATION_NAME"').count()

### Continuer le traitement des données
À partir d’ici, vous pouvez continuer à interroger et traiter les données.

In [ ]:

USE DATABASE CITIBIKE;

CREATE STAGE IF NOT EXISTS WEATHER_STAGE 
	URL = 's3://logbrain-datalake/datasets/weather-nyc-json/';

### Vérifier les fichiers présents dans le stage

In [ ]:
LS @WEATHER_STAGE;

### Charger le fichier CSV avec Snowpark
Nous pouvons utiliser Snowpark DataFrameReader pour lire le fichier JSON.

En utilisant l’option infer_schema = True, Snowflake déduira automatiquement le schéma à partir des types de données présents dans le fichier JSON, ce qui évite de devoir le définir manuellement.

In [ ]:
# Create a DataFrame that is configured to load data from the json file.
df = session.read.option("compression", "gzip").json('@WEATHER_STAGE/hourlyData-2018-1.json.gz')

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
df.dtypes

## Écrire le DataFrame dans une table Snowflake

Nous pouvons enregistrer le DataFrame dans une table appelée `weather_json` puis l’interroger en SQL.

In [ ]:
df.write.mode("overwrite").save_as_table("WEATHER_JSON")

### Écrire le DataFrame dans une table Snowflake

In [ ]:
-- Preview the newly created weather table
SELECT * from WEATHER_JSON;

### Relire la table dans Snowpark

In [ ]:
df = session.table("WEATHER_JSON")
df

## Parser la table JSON 

In [ ]:
USE DATABASE CITIBIKE;

CREATE TABLE IF NOT EXISTS weather as
select 
   $1:"coco"::STRING as "coco" ,
   $1:"country"::STRING as "country",
   $1:"dwpt"::FLOAT as "dwpt",
   $1:"ele$1ation"::STRING as "ele$1ation",
   $1:"icao"::STRING as "icao",
   $1:"latitude"::DECIMAL as "latitude",
   $1:"longitude"::DECIMAL as "longitude",
   $1:"name"::STRING as "name",
   $1:"obsTime"::TIMESTAMP as "obsTime",
   $1:"prcp"::STRING as "prcp" ,
   $1:"pres"::DECIMAL as "pres",
   $1:"region"::STRING as "region",
   $1:"rhum"::STRING as "rhum",
   $1:"snow"::STRING as "snow",
   $1:"station"::STRING as "station",
   $1:"temp"::DECIMAL "temp",
   $1:"timezone"::STRING as "timezone",
   $1:"tsun"::STRING as "tsun",
   $1:"wdir"::STRING as "wdir",
   $1:"weatherCondition"::STRING as "weatherCondition",
   $1:"wmo"::STRING as "wmo",
   $1:"wpgt"::STRING as "wpgt",
   $1:"wspd"::DECIMAL as "wspd"
 from WEATHER_JSON;


In [ ]:
-- Preview the newly created weather table
SELECT * from WEATHER;

### Relire la table dans Snowpark

In [ ]:
df = session.table("WEATHER")
df

### Créer un file format JSON

In [ ]:
format = session.sql("create or replace file format json_format type = 'JSON' strip_outer_array = true;").collect()

# Create a DataFrame that is configured to load data from the json file.
df = session.read.option("format_name", "json_format").json('@WEATHER_STAGE/hourlyData-2018-1.json.gz')

df

In [ ]:
df.write.mode("overwrite").save_as_table("WEATHER_JSON")

In [ ]:
select * from WEATHER_JSON;

In [ ]:
USE DATABASE CITIBIKE;

CREATE OR REPLACE TABLE  weather as
select 
   $1:"coco"::STRING as "coco" ,
   $1:"country"::STRING as "country",
   $1:"dwpt"::FLOAT as "dwpt",
   $1:"ele$1ation"::STRING as "ele$1ation",
   $1:"icao"::STRING as "icao",
   $1:"latitude"::DECIMAL as "latitude",
   $1:"longitude"::DECIMAL as "longitude",
   $1:"name"::STRING as "name",
   $1:"obsTime"::TIMESTAMP as "obsTime",
   $1:"prcp"::STRING as "prcp" ,
   $1:"pres"::DECIMAL as "pres",
   $1:"region"::STRING as "region",
   $1:"rhum"::STRING as "rhum",
   $1:"snow"::STRING as "snow",
   $1:"station"::STRING as "station",
   $1:"temp"::DECIMAL "temp",
   $1:"timezone"::STRING as "timezone",
   $1:"tsun"::STRING as "tsun",
   $1:"wdir"::STRING as "wdir",
   $1:"weatherCondition"::STRING as "weatherCondition",
   $1:"wmo"::STRING as "wmo",
   $1:"wpgt"::STRING as "wpgt",
   $1:"wspd"::DECIMAL as "wspd"
 from WEATHER_JSON;

In [ ]:
select * from weather;

### Nettoyage des ressources

In [ ]:
--  Database, table and stage created as part of this example
DROP TABLE IF EXISTS TRIPS;
DROP TABLE IF EXISTS WEATHER;
DROP TABLE IF EXISTS WEATHER_JSON;
DROP STAGE IF EXISTS CITIBIKE_STAGE;
DROP DATABASE IF EXISTS CITIKIE;

## Conclusion
Dans cet exemple, nous avons vu comment charger des fichiers CSV et JSON depuis un stage externe afin de traiter et interroger les données dans un notebook à l’aide de Snowpark.